In [ ]:
# !pip install paddleocr

In [ ]:
# !pip install paddlepaddle==2.6.0

In [ ]:
# !pip install ultralytics==8.1.3

In [ ]:
# !pip install opencv-python

In [ ]:
import cv2
import numpy as np
from paddleocr import PaddleOCR
from ultralytics import YOLO

In [ ]:
# Carrega o modelo YOLOV8 para classificação de imagens
model = YOLO('../src/runs/classify/train/weights/best.pt')

In [ ]:
# Carrega o modelo PaddleOCR para OCR
ocr = PaddleOCR(lang="pt")

In [ ]:

# Inicializa a webcam
cap = cv2.VideoCapture(0)

while True:
    # Captura um frame da webcam
    ret, frame = cap.read()

    # Detecta objetos no frame usando YOLOv8
    detections = model.detect(frame)

    # Itera sobre as detecções
    for detection in detections:
        # Verifica se o objeto é um documento de texto
        if detection.class_name == "document":
            # Realiza OCR no documento usando o PaddleOCR
            result = ocr.ocr(detection.image)

            # Imprime o texto reconhecido
            print(result[0][1])
        else:
            # Descreve a cena usando o modelo pre-treinado para imagens naturais
            caption = model.describe_image(detection.image)
            print(caption)

    # Exibe o frame
    cv2.imshow("Imagem", frame)

    # Verifica se o usuário pressionou a tecla 'q' para sair
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Libera a webcam
cap.release()
cv2.destroyAllWindows()

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import paddleocr

# Define os parâmetros de treinamento
learning_rate = 0.001
batch_size = 32
num_epochs = 20

# Carrega o dataset de imagens naturais
train_dataset = datasets.ImageFolder(
    "dataset/train",
    transform=transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    ),
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Cria o modelo de descrição de cena
model = paddleocr.SceneDescriptionModel()

# Define a função de perda
criterion = nn.CrossEntropyLoss()

# Define o otimizador
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Treina o modelo
for epoch in range(num_epochs):
    for i, data in enumerate(train_loader):
        # Obtém as entradas e rótulos
        inputs, labels = data

        # Ativa o modo de treinamento
        model.train()

        # Realiza um passo de forward propagation
        outputs = model(inputs)

        # Calcula a perda
        loss = criterion(outputs, labels)

        # Realiza um backward propagation
        loss.backward()

        # Atualiza os pesos
        optimizer.step()

        # Imprime o progresso do treinamento
        if i % 100 == 0:
            print(f"Epoch: {epoch}, Batch: {i}, Loss: {loss.item()}")

# Salva o modelo treinado
torch.save(model.state_dict(), "scene_description_model.pt")